# Wiki_Scraper_links
version date:12-04-2022

In [24]:
import wikipediaapi # pip3 install wikipedia-api
import pandas as pd
import numpy as np
import re
import bs4
import sys
import requests

# read .csv with wiki links (needs URL column )
file_path = 'spreadsheet_documents_dutch.csv'
documents_df = pd.read_csv(file_path)
doc_links = documents_df['url']

# filter for wiki links
doc_links = doc_links[doc_links.str.contains('nl.wikipedia')].tolist()

# remove dupicates 
doc_links = set(doc_links)

# check for number of documents
print('Number of documents: {}'.format(len(doc_links)))

Number of documents: 57


In [29]:
# empty dict for info
data = {'title_wiki':[], 'title_system':[], 'web_link':[]}

# connect to wiki API
wiki_wiki = wikipediaapi.Wikipedia(
        language='nl',
        extract_format=wikipediaapi.ExtractFormat.WIKI)


# wiki text extractor
def write_txt(web_link):
    """
    finds the text of wikipedia articles using bs4 and wikipediaapi
    
    web_link: A set() or list of links to Wikipedia
    """
    
    # get HTML File
    res = requests.get(web_link)
    res.raise_for_status()
    wiki = bs4.BeautifulSoup(res.text,'html.parser')
    
    # get title
    title_wiki = wiki.find(id='firstHeading').string
    title_system = re.subn("[-|*|\\s+|%]", '_', title_wiki)

    # save items to dict
    data['title_wiki'].append(title_wiki)
    data['title_system'].append(title_system[0])
    data['web_link'].append(web_link)
    
    # getting text 
    p_wiki = wiki_wiki.page(title_wiki)
    
    # document validity check
    if len(p_wiki.text) > 10:
        
#       # remove 'Zie ook\n' and 'Externe links'
        sep = 'Zie ook\n'
        sep_1 = 'Externe links'
        p_wiki = p_wiki.text
        p_wiki = p_wiki.split(sep, 1)[0]
        p_wiki = p_wiki.split(sep_1, 1)[0]
        
        
        
        # save document as .txt
        with open((str(title_system[0])+'.txt'), 'w') as f:   
                f.write(p_wiki)
            
    
    return data

In [30]:
%%time
sub = list(doc_links)[0:15]
# Getting requierments 
for i in sub:
    write_txt(i)

# Create data frame with titles
title_link = pd.DataFrame(data, columns=['title_wiki', 'title_system','web_link'])

title_link

CPU times: user 1.18 s, sys: 43.8 ms, total: 1.23 s
Wall time: 8.17 s


,title_wiki,title_system,web_link
0,Lijst van vindplaatsen van fossielen in Nederland,Lijst_van_vindplaatsen_van_fossielen_in_Nederland,https://nl.wikipedia.org/wiki/Lijst_van_vindpl...
1,Familie (biologie),Familie_(biologie),https://nl.wikipedia.org/wiki/Familie_(biologie)
2,Radiogolf,Radiogolf,https://nl.wikipedia.org/wiki/Radiogolf
3,Central Intelligence Agency,Central_Intelligence_Agency,https://nl.wikipedia.org/wiki/Central_Intellig...
4,Wim Helsen,Wim_Helsen,https://nl.wikipedia.org/wiki/Wim_Helsen
5,Chris Koolmees,Chris_Koolmees,https://nl.wikipedia.org/wiki/Chris_Koolmees
6,IALA Maritiem Betonningsstelsel,IALA_Maritiem_Betonningsstelsel,https://nl.wikipedia.org/wiki/IALA_Maritiem_Be...
7,Nederlandse Omroep Stichting,Nederlandse_Omroep_Stichting,https://nl.wikipedia.org/wiki/Nederlandse_Omro...
8,Kunstmatige intelligentie,Kunstmatige_intelligentie,https://nl.wikipedia.org/wiki/Kunstmatige_inte...
9,Koninklijk besluit,Koninklijk_besluit,https://nl.wikipedia.org/wiki/Koninklijk_besluit
